# 여러가지 방법의 모델 구축

## keras
1. sequential API

2. functional API 

3. class definition



## 배울 것

1. weight 저장방법
2. 모델 전체 저장방법 -> serializing model
    - weight
    - 모델 구조
    - model.compile() 로 정의한 학습 방법
    - optimizer, state

In [1]:
# Sequential

In [3]:
# Functional

In [5]:
# Class

# weight 저장방법


저장할 때는 동일한 방법으로 모델을 구현하고 해당 모델에 가중치를 로드해야한다.

# model 전체 저장방법

```

model.save('')
```

#### 반드시 디렉토리 형태로 저장된다.



```
model = keras.models.load_model('')
```

## 위의 모델 저장방법을 사용하는 경우, 모델 + 모델에 대한 설정까지 포함한다


- 모델의 아키텍처 (구조)
- 모델의 가중치 (내용물)
- 컴파일 정보 (구동방식)
- optimizer, 현재까지 학습한 상태 (현재의 위치)


## 저장 가능 형식은 Tensorflow SavedModel, Keras H5 형식

- 디폴트는 SavedModel 형식이다.



## SavedModel 형식이 사용자 정의 Object를 다루는 방법

- class name, call function, losses, weights, config를 저장한다.
- 사용자 정의 class가 코드에 존재하지 않아도 가능하다. (애초에 클래스 이름과 call function이 존재하기 때문에)
- 하지만, call function 만을 저장하기 때문에 학습 및 평가는 가능하지만 다른 method는 사용하지 못한다.


# Keras H5

H5 형식은 다음과 같이 저장하며

- 모델의 아키텍쳐
- 모델의 가중치
- 컴파일 정보


saved model과의 차이는 

- 사용자 정의 object를 저장하지 않는다는 것
- add_loss(), add_metric()으로 추가한 것은 저장되지 않는다.
- Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model.
> 즉, Functional model or a Sequential model 에 대해서만 사용이 가능하다.  
> 내부에서 subclass layer의 사용도 하지 못한다.


# 아키텍처 저장

SubClass 모델이 아닌 아래의 방식으로 구현하였을 때만 사용이 가능하다.
- 함수형
- Sequential API


```
get_config()


from_config()


tf.keras.models.model_to_json()


tf.keras.models.model_from_json()
```


# 모델의 가중치 값만 저장하거나 로딩하기

오직 가중치만 저장하고 가져오기만 가능하도록 하고싶을 수가 있다. 이럴 때가 있나 싶지만

1. 추론즉, 결과값 반환만을 수행하고 싶을 때 사용한다. 이 때는 학습이 필요하지 않기 때문에 compile option, optimizer들이 필요가 없다.
2. 전이학습을 진행하는 경우에 사용한다. 이 전 모델의 상태만 재사용하여 새로운 모델을 트레이닝하는 경우, 즉, 이전 모델의 compile option, optimizer가 필요하지 않지 않기 때문에 가중치만 가져와서 사용한다.


## 전이학습

> 기존의 데이터를 통해 구현한 문제 해결 방식들을 연관성이 존재하는 다른 문제에 적용할 수 있도록 하는 머신러닝의 한 방법

이 때, 가중치 값만 가져오기 떄문에 모델의 아키텍처는 구현이 되어있거나 선언되어있어야한다.

* 가중치가 존재하지 않는 relu, dropout, batchNormalization 과같은 layer가 있더라도 가중치의 값을 로딩할 수 있다.



### 메모리 상에서 가중치 전달

```python
# 해당 모델에서 weight를 가져온다.
model.get_weights()


#
new_model.set_weights(<weight>)

```


### 디스크에 가중치 저장, 로드

```python
model.save_weights(<save_format>)

```

모델을 저장하고 다시 로드할 때, 반드시 모델을 구현하는 방식 (subclass, functional API, sequential API)을 동일하게 사용해야한다.